In [ ]:
import os
import shutil

kaggle_token_path = '/content/drive/MyDrive/kaggle/kaggle.json'

colab_token_dir = '/root/.kaggle/'

if not os.path.exists(colab_token_dir):
    os.makedirs(colab_token_dir)

colab_token_path = os.path.join(colab_token_dir, 'kaggle.json')

shutil.copyfile(kaggle_token_path, colab_token_path)


'/root/.kaggle/kaggle.json'

In [ ]:
from google.colab import files

uploaded = files.upload()


!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.js/
!kaggle datasets download -d akash2907/bird-species-classification
!unzip bird-species-classification.zip


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.js/': No such file or directory
100% 1.37G/1.37G [00:10<00:00, 84.4MB/s]
100% 1.37G/1.37G [00:10<00:00, 142MB/s] 
Archive:  bird-species-classification.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_d

In [ ]:
import numpy as np
import pandas as pd
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Sequential

import numpy as np
from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
root_train = '/content/train_data/train_data'
root_test = '/content/test_data/test_data'

train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))

train = train_gen.flow_from_directory('/content/train_data/train_data',
                                      target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)
test = test_gen.flow_from_directory('/content/test_data/test_data',
                                    target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)

train.class_indices

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [ ]:
model = Sequential()
# Adding conv layer with input
model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))
# Normalizing the conv layer output
model.add(BatchNormalization())
# Selecting the max values
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropping the unwanted 20% of data
model.add(Dropout(0.2))
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
# Hiddern layers
model.add(Dense(62,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
# Output layer
model.add(Dense(16,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 12)      336       
                                                                 
 batch_normalization (BatchN  (None, 118, 118, 12)     48        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 12)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 12)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 24)        2616      
                                                                 
 batch_normalization_1 (Batc  (None, 57, 57, 24)       9

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='accuracy',patience=8)

In [21]:
model.fit(train,batch_size=8,validation_data=test,epochs=50,callbacks=early_stop)
# model.fit(train,batch_size=8,validation_data=test,epochs=50)

Epoch 1/50
19/19 [==============================] - 140s 8s/step - loss: 0.2148 - accuracy: 0.9533 - val_loss: 3.8919 - val_accuracy: 0.2293
Epoch 2/50
19/19 [==============================] - 129s 7s/step - loss: 0.2328 - accuracy: 0.9267 - val_loss: 3.5593 - val_accuracy: 0.2611
Epoch 3/50
19/19 [==============================] - 129s 7s/step - loss: 0.3113 - accuracy: 0.9067 - val_loss: 3.5995 - val_accuracy: 0.2484
Epoch 4/50
19/19 [==============================] - 128s 7s/step - loss: 0.3633 - accuracy: 0.8867 - val_loss: 4.3490 - val_accuracy: 0.2102
Epoch 5/50
19/19 [==============================] - 129s 7s/step - loss: 0.3355 - accuracy: 0.9000 - val_loss: 3.9279 - val_accuracy: 0.1783
Epoch 6/50
19/19 [==============================] - 127s 7s/step - loss: 0.2816 - accuracy: 0.9133 - val_loss: 3.9392 - val_accuracy: 0.2548
Epoch 7/50
19/19 [==============================] - 127s 7s/step - loss: 0.3874 - accuracy: 0.8933 - val_loss: 3.5081 - val_accuracy: 0.2803
Epoch 8/50
19

In [22]:
model.save('birds.h5')

In [23]:
img1 = image.load_img('/content/test_data/test_data/bonegl/DSC_4587.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['blasti', 'bonegl', "brhkyt", "cbrtsh", "cmnmyn", "gretit", "hilpig", "himbul",
          "himgri", "hsparo", "indvul", "jglowl", "lbicrw", "mgprob", "rebimg", "wcrsrt"]
print(output[pred])

1/1 [==============================] - 0s 331ms/step
8
himgri


In [24]:
model.predict(test)

20/20 [==============================] - 61s 3s/step


array([[4.2168428e-03, 3.3613509e-01, 1.1956173e-03, ..., 3.2792783e-03,
        4.2430093e-03, 1.2514754e-03],
       [8.4055975e-02, 2.1066605e-03, 2.3872693e-01, ..., 1.1639467e-04,
        1.0248720e-04, 4.4777509e-04],
       [1.0550067e-04, 2.9806658e-03, 2.9869629e-05, ..., 1.6846597e-01,
        1.2477988e-01, 1.4687029e-01],
       ...,
       [9.1655958e-05, 3.3461191e-02, 1.3704671e-04, ..., 6.3324240e-03,
        2.4995755e-04, 5.5371062e-03],
       [1.0233947e-06, 6.1710698e-10, 9.1399561e-06, ..., 1.5542606e-07,
        3.5104435e-12, 5.8541694e-08],
       [9.7594786e-08, 1.9772041e-07, 4.1196570e-05, ..., 3.9360835e-05,
        7.3145857e-05, 1.9017808e-03]], dtype=float32)

In [25]:
test_loss, test_accuracy = model.evaluate(test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

20/20 [==============================] - 60s 3s/step - loss: 5.3699 - accuracy: 0.2229
Test Loss: 5.36991548538208
Test Accuracy: 0.2229299396276474
